In [ ]:
# default_exp fault_generators

# Fault generators

> Collection of generators to produce faulty circuits.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
from qsam.circuit import Circuit
import numpy as np
import itertools as it

A faulty circuit is an "empty" circuit of the same size as a reference circuit which has "fault" gates at some circuit locations. Fault gates are just regular gates (X,Y,Z,..) just with the difference that these are executed in the simulation **after** the gates in a tick of the reference circuit. Thus, a fault circuit is always defined with respect to a reference circuit, i.e. with same number of qubits and ticks as the reference circuit. Here an example:

In [ ]:
ref_circ = Circuit([{"H":{0,1,2}}, {"CNOT": {(0,1), (0,2)}}, {"measure":{0,1,2}}])
f_circ = Circuit([dict()] * 3)
f_circ[1] = {"X": {0,2}}

print(ref_circ)
print(f_circ)

0: {'H': {0, 1, 2}}
1: {'CNOT': {(0, 1), (0, 2)}}
2: {'measure': {0, 1, 2}}
0: {}
1: {'X': {0, 2}}
2: {}


The fault generator implements a function to populate the fault circuit (randomly) due to some specified parameters. For now, we only implement a generator for depolarizing circuit-level noise, i.e. equal probability of errors on each given circuit element. For now we also only consider gates as circuit elements (1-qubit, 2-qubit, measurements). In the future, we will also implement a way to place idle and cross-talk noise.

In [ ]:
#export

ONE_QUBIT_FAULTS = ["X", "Y", "Z"]
TWO_QUBIT_FAULTS = list(it.product(ONE_QUBIT_FAULTS + ["I"], repeat=2))
TWO_QUBIT_FAULTS.remove(("I","I"))

class Depolar:
    """Fault circuit generator for depolarizing circuit-level noise"""
    
    def __init__(self, n_ticks):
        self.n_ticks = n_ticks
        
    def select_fault_locs(self, partitions, probs):
        """Select fault locs by random number < p for elements in partitions"""
        return [loc for locs,p in zip(partitions, probs) for loc in locs if np.random.random() < p]
    
    def generate(self, partitions, ps_or_ws):
        """Place `faults` in empty circuit at circuit locations `fault_locs`"""
        
        fault_circuit = Circuit([{} for _ in range(self.n_ticks)])
        fault_locs = self.select_fault_locs(partitions, ps_or_ws)
        
        for (tick_index, qubit) in fault_locs:
            if type(qubit) == int:
                f_gate = np.random.choice(ONE_QUBIT_FAULTS)
                qb_set = fault_circuit[tick_index].get(f_gate, set())
                qb_set.add(qubit)
                fault_circuit[tick_index][f_gate] = qb_set
            elif type(qubit) == tuple:
                f_gates = TWO_QUBIT_FAULTS[np.random.choice(len(TWO_QUBIT_FAULTS))]
                for f_gate, qubit_i in zip(f_gates, qubit):
                    if f_gate != "I":
                        qb_set = fault_circuit[tick_index].get(f_gate, set())
                        qb_set.add(qubit_i)
                        fault_circuit[tick_index][f_gate] = qb_set
        return fault_circuit

In [ ]:
partitions = [[(0,1),(1,3),(5,2)],[(0,(2,3)), (2,(4,3)), (3,(5,4))]]
ps = [0.4,0.2]

fault_gen = Depolar(n_ticks=6)
print(fault_gen.generate(partitions, ps))

0: {}
1: {}
2: {}
3: {}
4: {}
5: {'Y': {2}}


For the `SubsetSampler` we need to select faults of a specific amount (i.e. weight) from the partitions. Thus, we create a new class which inherits from `Depolar` and overwrites its `select_fault_locs` function.

In [ ]:
#export
class DepolarSS(Depolar):
    def select_fault_locs(self, partitions, weights):
        """Select fault locs by random choice of w elements per partition"""
        return [par[i] for par,w in zip(partitions, weights) for i in np.random.choice(len(par),w,replace=False)]

In [ ]:
ws = [0,2]

fault_genSS = DepolarSS(n_ticks=6)
print(fault_genSS.generate(partitions, ws))

0: {'X': {2, 3}}
1: {}
2: {'X': {4}, 'Y': {3}}
3: {}
4: {}
5: {}
